# 如何缓存聊天模型的响应

:::info 前置条件

本指南假设您已熟悉以下概念：
- [聊天模型](/docs/concepts/chat_models)
- [LLMs](/docs/concepts/text_llms)

:::

LangChain 为 [聊天模型](/docs/concepts/chat_models) 提供了一个可选的缓存层。这主要有两个好处：

- 如果您经常多次请求相同的完成内容，它可以通过减少对 LLM 提供商的 API 调用次数来为您节省费用。这在应用程序开发期间尤其有用。
- 它可以通过减少对 LLM 提供商的 API 调用次数来加快您的应用程序速度。

本指南将向您展示如何在您的应用程序中启用此功能。

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />


In [1]:
# | output: false
# | echo: false

import os
from getpass import getpass

from langchain_openai import ChatOpenAI

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

llm = ChatOpenAI()

In [2]:
# <!-- ruff: noqa: F821 -->
from langchain_core.globals import set_llm_cache

## 内存缓存

这是一个临时缓存，将模型调用存储在内存中。当您的环境重启时，它将被清除，并且不会在进程之间共享。

In [ ]:
%%time
from langchain_core.caches import InMemoryCache

set_llm_cache(InMemoryCache())

# 第一次调用时，尚未缓存，因此会花费更长时间
llm.invoke("讲个笑话")

CPU times: user 645 ms, sys: 214 ms, total: 859 ms
Wall time: 829 ms


AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything!", response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 11, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-b6836bdd-8c30-436b-828f-0ac5fc9ab50e-0')

In [ ]:
%%time
# 第二次调用时，已缓存，因此速度更快
llm.invoke("讲个笑话")

CPU times: user 822 µs, sys: 288 µs, total: 1.11 ms
Wall time: 1.06 ms


AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything!", response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 11, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-b6836bdd-8c30-436b-828f-0ac5fc9ab50e-0')

## SQLite 缓存

此缓存实现使用 `SQLite` 数据库存储响应，并且在进程重启后仍然有效。

In [5]:
!rm .langchain.db

In [ ]:
# 我们可以对 SQLite 缓存执行相同的操作
from langchain_community.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [ ]:
%%time
# 第一次调用时，尚未缓存，因此会花费更长时间
llm.invoke("讲个笑话")

CPU times: user 9.91 ms, sys: 7.68 ms, total: 17.6 ms
Wall time: 657 ms


AIMessage(content='Why did the scarecrow win an award? Because he was outstanding in his field!', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 11, 'total_tokens': 28}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-39d9e1e8-7766-4970-b1d8-f50213fd94c5-0')

In [ ]:
%%time
# 第二次调用时，已缓存，因此速度更快
llm.invoke("讲个笑话")

CPU times: user 52.2 ms, sys: 60.5 ms, total: 113 ms
Wall time: 127 ms


AIMessage(content='Why did the scarecrow win an award? Because he was outstanding in his field!', id='run-39d9e1e8-7766-4970-b1d8-f50213fd94c5-0')

## 下一步

您现在已经了解了如何缓存模型响应以节省时间和金钱。

接下来，请查看本节中关于聊天模型的其他操作指南，例如[如何让模型返回结构化输出](/docs/how_to/structured_output)或[如何创建您自己的自定义聊天模型](/docs/how_to/custom_chat_model)。